<a href="https://colab.research.google.com/github/CSID-DGU/2021-1-OSSP1-Debugger-4/blob/Fid-score/fid_example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 라이브러리 설정
from __future__ import absolute_import, division, print_function
import os
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize




Mounted at /content/drive


In [ ]:

def calculate_frechet_distance(images1,images2, eps=1e-6):
    # prepare the inception v3 model
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)

    assert mu1.shape == mu2.shape, "Training and test mean vectors have different lengths"
    assert sigma1.shape == sigma2.shape, "Training and test covariances have different dimensions"

    diff = mu1 - mu2

    # product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = "fid calculation produces singular product; adding %s to diagonal of cov estimates" % eps
        warnings.warn(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean
#-------------------------------------------------------------------------------
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)


In [ ]:

from __future__ import absolute_import, division, print_function
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import random
import cv2
import gc
import glob
import tensorflow
from tensorflow.keras.layers import Reshape


# define two collections of images
images1 = glob.glob('./drive/My Drive/Colab Notebooks/images.png')
images2 = glob.glob('./drive/My Drive/Colab Notebooks/images2.png')

images1 = images1.reshape((1912, 64, 64, 3))
images2 = images1.reshape((1912, 64, 64, 3))
#print('Prepared', images1.shape, images2.shape)


# convert integer to floating point values
#images1 = images1.astype('float32')
#images2 = images2.astype('float32')

# resize images
#images1 = scale_images(images1, (299,299,3))
#images2 = scale_images(images2, (299,299,3))0
#print('Scaled', images1.shape, images2.shape)
# pre-process images

#images1 = preprocess_input(images1)
#images2 = preprocess_input(images2)

# fid between images1 and images2
fid = calculate_frechet_distance(images1, images2)
print('FID (different): %.3f' % fid)

AttributeError: ignored